<a href="https://colab.research.google.com/github/tmendonca28/pytorch-trial/blob/master/CNNs_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks


*   Additional convolutions and pooling layers before feedforward neural network
*  **Input** -> **Additional Layers for C and P** -> **Fully Connected** -> **Output**



# One Convolutional Layer
* The 'torch light' is called a filter or kernel. (of size K, K)
* It maps a single patch to a number known as the feature map.
* A convolution is then a mapping from input to a bunch of numbers in the feature maps.
* For *n* number of kernels we get *n* number of feature maps.
* 2 operations are done per patch: element-wise mult. followed by summation.

# Model A
* 2 Convolutional Layer
  * Same padding (same output size)
* 2 Max pooling layers
* 1 Fully Connected Layer

In [0]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [0]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)
test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

In [0]:
print(train_dataset.data.size())

torch.Size([60000, 28, 28])


In [0]:
print(train_dataset.targets.size())

torch.Size([60000])


In [0]:
print(test_dataset.data.size())

torch.Size([10000, 28, 28])


In [0]:
print(test_dataset.targets.size())

torch.Size([10000])


In [0]:
batch_size = 100
n_iters = 3000
num_epochs = n_iters/(len(train_dataset)/batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
class CNNModel(nn.Module):
  def __init__(self):
    super(CNNModel, self).__init__()
    
    # Convolution 1
    # in_channels is 1 because its a single grayscale image; out_channels=16 means 16 feature maps
    self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
    self.relu1 = nn.ReLU()
    
    # Max pool 1
    self.maxpool1 = nn.MaxPool2d(kernel_size=2)
    
    # Convolution 2
    self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
    self.relu2 = nn.ReLU()
    
    # Max pool 2
    self.maxpool2 = nn.MaxPool2d(kernel_size=2)
    
    # Fully-connected layer
    self.fc1 = nn.Linear(32 * 7 * 7,10)
    
    
  def forward(self, x):
    # Convolution 1
    out = self.cnn1(x)
    out = self.relu1(out)
    
    # Max pool 1
    out = self.maxpool1(out)
    
    # Convolution 2
    out = self.cnn2(out)
    out = self.relu2(out)
    
    # Max pool 2
    out = self.maxpool2(out)
    
    # Resize
    # Original size: (100, 32, 7, 7)
    # out.size(0): 100
    # New out size: (100, 32 * 7 * 7)
    
    out = out.view(out.size(0), -1)
    
    # Linear function (readout)
    out = self.fc1(out)
    
    return out
  
model = CNNModel()
criterion = nn.CrossEntropyLoss()

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [0]:
print(model.parameters())
print(len(list(model.parameters())))

print(list(model.parameters())[0].size())
print(list(model.parameters())[1].size())
print(list(model.parameters())[2].size())
print(list(model.parameters())[3].size())
print(list(model.parameters())[4].size())
print(list(model.parameters())[5].size())

<generator object Module.parameters at 0x7fc77cbf7a40>
6
torch.Size([16, 1, 5, 5])
torch.Size([16])
torch.Size([32, 16, 5, 5])
torch.Size([32])
torch.Size([10, 1568])
torch.Size([10])


In [0]:
# Training the model
iter = 0
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    # Load images as Variable
    images = Variable(images)
    labels = Variable(labels)
    
    # Clear gradients w.r.t params
    optimizer.zero_grad()
    
    # Forward pass to get output
    outputs = model(images)
    
    # Calculate loss
    loss = criterion(outputs, labels)
    
    # Get gradients w.r.t params
    loss.backward()
    
    # Update params
    optimizer.step()
    
    iter+=1
    
    if iter%500 == 0:
      # Calculate Accuracy (every 500 iterations)
      correct = 0
      total = 0
      
      # Iterate through the test dataset
      for images, labels in test_loader:
        # Load images into a torch Variable
        images = Variable(images)
        
        # Forward pass only to get output
        outputs = model(images)
        
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs.data, 1)
        
        # Total nuber of labels
        total += labels.size(0)
        
        # Total correct predictions
        correct += (predicted == labels).sum()
        
      accuracy = 100 * correct/total
    
      # Print Loss
      print('Iteration:{}. Loss:{}. Accuracy: {}.'.format(iter, loss.item(), accuracy))

Iteration:500. Loss:0.24468038976192474. Accuracy: 90.
Iteration:1000. Loss:0.2804596424102783. Accuracy: 91.
Iteration:1500. Loss:0.1128520742058754. Accuracy: 94.
Iteration:2000. Loss:0.2720423638820648. Accuracy: 95.
Iteration:2500. Loss:0.08201614022254944. Accuracy: 96.
Iteration:3000. Loss:0.04539855942130089. Accuracy: 96.
